### PromptTemplate 
* [PromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html#langchain_core.prompts.prompt.PromptTemplate)
* [ChatPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html#langchain_core.prompts.chat.ChatPromptTemplate)
* [ChatMessagePromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatMessagePromptTemplate.html#langchain_core.prompts.chat.ChatMessagePromptTemplate)
* [FewShotPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.few_shot.FewShotPromptTemplate.html#langchain_core.prompts.few_shot.FewShotPromptTemplate)
* PartialPrompt

In [ ]:
# poetry add python-dotenv langchain langchain-openai

In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


##### 1) PromptTemplate 의 from_template() 함수 사용
* 주로 LLM(텍스트 완성형 모델, ex. Ollama, GPT-3.5)과 함께 사용
* 하나의 문자열 프롬프트를 생성

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from pprint import pprint

template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="openai/gpt-oss-120b",  # Spring AI와 동일한 모델
    temperature=0.7
)

chain = prompt_template | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3})
pprint(response)

BadRequestError: Error code: 400 - {'error': {'message': 'Organization has been restricted. Please reach out to support if you believe this was in error.', 'type': 'invalid_request_error', 'code': 'organization_restricted'}}

##### 2) PromptTemplate 결합하기
* 동일한 Prompt 패턴을 사용하지만 여러 개의 질문을 작성해서 LLM을 실행할 수도 있습니다.

In [5]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)
combined_prompt.format(model_name="ChatGPT", count=3, language="한국어")
print(combined_prompt)

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"한국어"})

pprint(response)

input_variables=['count', 'language', 'model_name'] input_types={} partial_variables={} template='{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요.\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요.'
('**ChatGPT 모델의 학습 원리 (3문장)**  \n'
 '1. 방대한 텍스트 데이터를 이용해 토큰(단어·문자)의 순서를 예측하는 **자기지도 학습**을 수행합니다.  \n'
 '2. Transformer 구조의 **다중 헤드 어텐션**을 통해 문맥을 장기적으로 파악하고, 각 레이어가 점진적으로 더 높은 수준의 '
 '언어 표현을 학습합니다.  \n'
 '3. 사전 학습된 모델을 특정 작업에 맞게 **미세조정(fine‑tuning)**하거나 **프롬프트 엔지니어링**으로 원하는 출력을 '
 '유도합니다.  \n'
 '\n'
 '---\n'
 '\n'
 '### ChatGPT 모델의 장점 요약\n'
 '- **다양한 분야에 적용 가능**: 질문‑응답, 글쓰기 보조, 코드 생성 등 여러 도메인에서 높은 품질의 텍스트를 생성한다.  \n'
 '- **문맥 이해 능력**: 수천 토큰에 이르는 긴 문맥을 유지하면서 일관된 대화를 이어갈 수 있다.  \n'
 '- **빠른 적응성**: 프롬프트만 바꿔도 새로운 주제나 스타일에 즉시 대응할 수 있어 별도 재학습 없이도 활용도가 높다.  \n'
 '- **언어 다중성**: 여러 언어를 동시에 학습해 다국어 환경에서도 비교적 자연스러운 번역·대화를 제공한다.  \n'
 '\n'
 '---\n'
 '\n'
 '### ChatGPT와 비슷한 AI 모델 (한국어 명칭)\n'
 '- **LLaMA (라마) – Meta**  \n'
 '- **Gemini (제미니) – Go

#### PromptTemplate 의 파라미터를 배열 형태로 하여 여러개 사용하는 경우

In [6]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 2},
    {"model_name": "Gemma", "count": 3},
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions]
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

for prompt in formatted_prompts:
    print(type(prompt), prompt)
    response = llm.invoke(prompt)
    pprint(response.content)

['GPT-4 모델의 학습 원리를 2 문장으로 한국어로 답변해 주세요.', 'Gemma 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.']
<class 'str'> GPT-4 모델의 학습 원리를 2 문장으로 한국어로 답변해 주세요.
('GPT‑4는 방대한 텍스트 데이터를 이용해 다음에 올 토큰을 예측하도록 하는 자기지도 학습으로 사전학습을 진행하고, 이후 인간이 만든 '
 '예시와 보상 모델을 활용한 강화학습(RLHF)으로 원하는 답변 품질을 높이는 미세조정을 수행합니다. 이렇게 두 단계의 학습 과정을 거쳐 '
 '다양한 언어 패턴과 문맥을 이해하고 생성할 수 있게 됩니다.')
<class 'str'> Gemma 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.
('Gemma 모델은 대규모 텍스트 데이터를 사용해 토큰 단위의 다음 단어를 예측하도록 사전 학습됩니다.  \n'
 '학습 과정에서는 트랜스포머 아키텍처를 기반으로 자기‑주의 메커니즘을 활용해 문맥 정보를 효율적으로 통합합니다.  \n'
 '그 후, 특정 작업에 맞게 미세 조정(fine‑tuning)하거나 프롬프트 엔지니어링을 적용해 실제 응용에 활용됩니다.')


##### 2) ChatPromptTemplate
* Tuple 형태의 system, user, assistant 메시지 지원
* 여러 개의 메시지를 조합하여 LLM에게 전달 가능
* 간결성과 가독성이 높고 단순한 구조

In [9]:
# 2-튜플 형태의 메시지 목록으로 프롬프트 생성 (type, content)

from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    # role, message
    ("system", "This system is an expert in answering questions about {topic}. Please provide clear and detailed explanations."),
    ("human", "{model_name} 모델의 학습 원리를 설명해 주세요."),
])

messages = chat_prompt.format_messages(topic="AI", model_name="ChatGPT")
print(messages)

# 생성한 메시지를 바로 주입하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

print(type(response))
print(response.content)

[SystemMessage(content='This system is an expert in answering questions about AI. Please provide clear and detailed explanations.', additional_kwargs={}, response_metadata={}), HumanMessage(content='ChatGPT 모델의 학습 원리를 설명해 주세요.', additional_kwargs={}, response_metadata={})]
<class 'langchain_core.messages.ai.AIMessage'>
## ChatGPT 모델은 어떻게 학습되나요?  

아래에서는 **데이터 준비 → 모델 구조 → 사전 학습 → 지도 학습(슈퍼바이즈드 파인튜닝) → 인간 피드백을 활용한 강화 학습(RLHF)** 의 순서대로, 각 단계에서 어떤 일들이 일어나는지 자세히 살펴보겠습니다.

---

## 1. 데이터 준비 (Corpus Collection & Tokenization)

| 단계 | 내용 |
|------|------|
| **데이터 수집** | 웹 페이지, 책, 논문, 뉴스, 포럼, 코드 저장소 등 다양한 출처에서 수십 테라바이트 규모의 텍스트를 수집합니다. 개인정보, 불법·유해 콘텐츠는 필터링하고, 저작권 문제를 최소화하기 위해 공개된 데이터와 라이선스가 허용된 데이터를 주로 사용합니다. |
| **전처리** | HTML 태그 제거, 중복 문장 삭제, 언어 식별, 문장 정규화(공백, 특수 문자 정리) 등을 수행합니다. |
| **토크나이징** | 텍스트를 **Byte‑Pair Encoding (BPE)** 혹은 **SentencePiece**와 같은 서브워드 토크나이저로 변환합니다. <br>예: “ChatGPT” → `['Chat', '##G', '##PT']` (BPE) <br>이 과정을 통해 어휘 집합(vocabulary) 크기를 30k~100k 정도로 제한하고, 희귀 단어도 서브워드 조합으로 표

In [10]:
# 체인을 생성하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain = chat_prompt | llm | StrOutputParser()

response = chain.invoke({"topic":"AI", "model_name":"ChatGPT"})
print(type(response))
print(response)

<class 'str'>
## ChatGPT 모델의 학습 원리  
(한국어 + 가능한 한 상세하게)

---

### 1. 전체 흐름
1. **데이터 수집·전처리** → 2. **사전 학습(Pre‑training)** → 3. **미세 조정(Fine‑tuning)** → 4. **보상 학습(Reinforcement Learning with Human Feedback, RLHF)** → 5. **배포·서비스**  

아래에서 각 단계가 어떻게 이루어지는지, 핵심 원리와 기술을 차례대로 살펴보겠습니다.

---

## 2. 데이터와 토크나이저

| 단계 | 내용 |
|------|------|
| **데이터** | 웹 페이지, 책, 논문, 포럼, 대화 로그 등 수십억 토큰 규모의 텍스트. 저작권·프라이버시를 고려해 필터링하고, 품질이 낮은(스팸·광고·불법) 텍스트는 제외. |
| **토크나이저** | **Byte‑Pair Encoding (BPE)** 혹은 **SentencePiece** 기반 서브워드 토크나이저 사용. <br>예: “ChatGPT” → `['Chat', '##GP', '##T']`. <br>이 방식은 어휘 크기를 수만~수십만 개 정도로 제한하면서도 거의 모든 문자열을 표현 가능하게 함. |
| **입력 포맷** | 토큰 시퀀스 앞에 **특수 토큰**(예: ``)을 삽입하고, **포지셔널 인코딩**을 더해 모델이 토큰 순서를 인식하도록 함. |

---

## 3. 사전 학습 (Pre‑training)

### 3.1 모델 아키텍처 – Transformer Decoder
- **층(Layer)**: 여러 개의 **self‑attention** 블록 + **feed‑forward** 네트워크 (FFN).  
- **Self‑Attention**: 각 토큰이 시퀀스 전체(앞쪽 토큰 포함)와 얼마나 연관되는지를 학습.  
- **멀티‑헤드**: 여러 독립적인 attention 헤드가 서로 다른 관계(문법, 의미, 장거리 의존성 등)를 동시에 학습.  


#### 3) ChatPromptTemplate
* SystemMessagePromptTemplate와 HumanMessagePromptTemplate 클래스 사용
* 객체 지향적 접근 - Message 객체를 독립적으로 생성 가능
* 여러 조건에 따라 다른 시스템 메시지 선택

```python
if user_is_beginner:
    system_message = SystemMessagePromptTemplate.from_template("초보자를 위한 설명: {topic}")
else:
    system_message = SystemMessagePromptTemplate.from_template("전문가를 위한 상세 분석: {topic}")
```

In [11]:
# ChatMessagePromptTemplate 활용

from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    ChatMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "You are an AI expert in {topic}. Please provide clear and detailed explanations."
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "This is an example answer about {topic}."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="What is deep learning?")

# LLM 호출
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

# 결과 출력
print(response.content)

## Deep Learning – A Comprehensive Overview

### 1. What Is Deep Learning?
Deep learning (DL) is a sub‑field of **machine learning (ML)** that focuses on algorithms called **artificial neural networks** (ANNs) with many layers—hence the term *deep*. These networks automatically learn hierarchical representations of data, allowing them to perform tasks such as image recognition, natural‑language understanding, speech synthesis, and more, often surpassing traditional ML methods.

### 2. How It Relates to Other AI Concepts
| Concept | Scope | Typical Techniques | Role of Deep Learning |
|---------|-------|--------------------|-----------------------|
| **Artificial Intelligence (AI)** | Broad goal of creating machines that exhibit intelligent behavior. | Rule‑based systems, search algorithms, planning, ML, DL, robotics. | DL is one of the most powerful modern approaches for perception‑heavy tasks. |
| **Machine Learning (ML)** | Subset of AI that learns patterns from data without explicit

#### ChatMessagePromptTemplate는 여러 종류의 메시지(시스템, 인간, AI)를 조합하여 복잡한 프롬프트를 생성할 때 유용합니다.
* SystemMessagePromptTemplate: 이 템플릿은 AI 모델에게 역할을 부여하거나 전반적인 규칙을 설정하는 시스템 메시지를 만듭니다. 위의 예시에서는 "번역을 도와주는 유용한 도우미"라는 역할을 지정합니다.
* HumanMessagePromptTemplate: 이 템플릿은 사용자의 질문이나 요청을 담는 인간 메시지를 만듭니다. 아래의 예시에서는 번역할 텍스트를 입력받습니다.
* ChatPromptTemplate.from_messages: 이 클래스 메서드는 시스템 메시지, 인간 메시지 등 여러 종류의 MessagePromptTemplate 객체들을 리스트로 받아 하나의 채팅 프롬프트 템플릿으로 통합합니다.
* format_messages: 이 메서드는 정의된 템플릿에 실제 값을 채워 넣어 [SystemMessage, HumanMessage] 형태의 리스트를 반환합니다. 이 리스트는 채팅 모델(Chat Model) 에 바로 전달될 수 있습니다.

In [ ]:
# 필요한 라이브러리 임포트
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# 1. SystemMessagePromptTemplate와 HumanMessagePromptTemplate 생성
# SystemMessagePromptTemplate는 모델의 페르소나 또는 기본 지침을 설정합니다.
system_template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# HumanMessagePromptTemplate는 사용자로부터 받는 입력 프롬프트를 정의합니다.
human_template = "{text_to_translate}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# 2. ChatPromptTemplate 생성
# 위에서 만든 두 템플릿을 리스트로 묶어 ChatPromptTemplate을 만듭니다.
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# 3. 프롬프트 포맷팅
# chat_prompt_template.format_messages()를 사용하여 최종 메시지 리스트를 생성합니다.
# 이 함수는 딕셔너리 형태의 입력 변수를 받습니다.
formatted_prompt = chat_prompt_template.format_messages(
    input_language="English",
    output_language="Korean",
    text_to_translate="I love programming."
)

# 4. 결과 출력
print(formatted_prompt)

# LLM 호출
response = llm.invoke(formatted_prompt)

# 결과 출력
print(response.content)

[SystemMessage(content='You are a helpful assistant that translates English to Korean.', additional_kwargs={}, response_metadata={}), HumanMessage(content='I love programming.', additional_kwargs={}, response_metadata={})]
나는 프로그래밍을 좋아해요.


: 

##### 4) FewShotPromptTemplate
* FewShotPromptTemplate은 모델이 특정 형식을 따르게 하거나, 일관된 응답을 생성하도록 유도할 때 유용합니다.
* 도메인 지식이 필요하거나, AI가 오답을 줄이고 더 신뢰할 만한 답변을 생성하도록 해야 할 때 효과적입니다.

##### 4-1) PromptTemplate을 사용하지 않는 경우

In [19]:
# PromptTemplate을 사용하지 않는 경우
from langchain_openai import ChatOpenAI

# model
#llm = ChatOpenAI(model="gpt-3.5-turbo")

# chain 실행
result = llm.invoke("태양계의 행성들을 간략히 정리해 주세요.")

print(type(result))
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
태양계에는 8개의 행성이 있습니다. 

1.  수성: 태양과 가장 가까운 행성으로, 표면이 암석으로 구성되어 있고 극도로 높은 온도와 낮은 온도가 반복됩니다.
2.  금성: 태양계에서 두 번째로 가까운 행성으로, 두꺼운 대기로 인해 극심한 온실 효과가 발생하여 매우 뜨겁습니다.
3.  지구: 우리가 사는 행성으로, 물과 대기가 있어 생명체가 존재할 수 있습니다.
4.  화성: 태양계에서 네 번째로 가까운 행성으로, 붉은색의 모래사막으로 덮여 있고 물과 생명체의 존재 가능성이 있습니다.
5.  목성: 태양계에서 가장 큰 행성으로, 가스 거인이며 강력한 자기장과 수많은 위성을 가지고 있습니다.
6.  토성: 태양계에서 두 번째로 큰 행성으로, 가스 거인이며 아름다운 고리를 가지고 있습니다.
7.  천왕성: 태양계에서 일곱 번째로 가까운 행성으로, 가스 거인이며 자전축이 기울어져 있어 극단적인 기후 변화를 경험합니다.
8.  해왕성: 태양계에서 가장 먼 행성으로, 가스 거인이며 강한 바람과 극적인 기후 변화를 가지고 있습니다.

이러한 행성들은 각각 고유한 특징과 성질을 가지고 있으며, 태양계에서 중요한 역할을 합니다.


##### 4-2) FewShotChatMessagePromptTemplate 사용하는 경우

In [12]:
# FewShotChatMessagePromptTemplate 사용하는 경우
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "input": "뉴턴의 운동 법칙을 요약해 주세요.",
        "output": """### 뉴턴의 운동 법칙
1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.
2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.
3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다."""
    },
    {
        "input": "지구의 대기 구성 요소를 알려주세요.",
        "output": """### 지구 대기의 구성
- **질소 (78%)**: 대기의 대부분을 차지합니다.
- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.
- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.
- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다."""
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
chain = final_prompt | llm
print(chain)

# 테스트 실행
result = chain.invoke({"input": "태양계의 행성들을 간략히 정리해 주세요."})
print(result.content)

first=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다.'), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': '뉴턴의 운동 법칙을 요약해 주세요.', 'output': '### 뉴턴의 운동 법칙\n1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.\n2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.\n3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다.'}, {'input': '지구의 대기 구성 요소를 알려주세요.', 'output': '### 지구 대기의 구성\n- **질소 (78%)**: 대기의 대부분을 차지합니다.\n- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.\n- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.\n- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다.'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[

#### 5-1) PartialPrompt 
* 프롬프트를 더 동적으로 활용할 수 있으며, AI 응답을 더 일관성 있게 조정 가능함

In [13]:
from datetime import datetime
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# 프롬프트 템플릿 정의 (부분 변수 적용)
prompt = PromptTemplate(
    template="{season}에 일어나는 대표적인 지구과학 현상은 {phenomenon}이 맞나요? {season}에 주로 발생하는 지구과학 현상을 3개 알려주세요",
    input_variables=["phenomenon"],  # 사용자 입력 필요
    partial_variables={"season": get_current_season()}  # 동적으로 계절 값 할당
)

# OpenAI 모델 초기화
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

# 특정 계절의 현상 질의
query = prompt.format(phenomenon="태풍 발생")
result = llm.invoke(query)


# 결과 출력
print(f" 프롬프트: {query}")
print(f" 모델 응답: {result.content}")

 프롬프트: 가을에 일어나는 대표적인 지구과학 현상은 태풍 발생이 맞나요? 가을에 주로 발생하는 지구과학 현상을 3개 알려주세요
 모델 응답: 가을에 일어나는 지구과학 현상은 지역에 따라 차이가 있지만, **태풍(열대 저기압)** 역시 가을에 활발히 발생하는 대표적인 현상 중 하나입니다. 특히 동아시아·동북아 지역에서는 8월 말부터 10월까지 태풍이 가장 많이 발생하고, 가을이 되면 남쪽에서 북쪽으로 이동하면서 한반도·일본 등에 큰 영향을 미칩니다. 따라서 “가을에 일어나는 대표적인 지구과학 현상”으로 **‘태풍 발생’**을 꼽는 것이 충분히 타당합니다.

그 외에도 가을에 주목할 만한 지구과학 현상 3가지를 아래와 같이 소개합니다.

---

## 1️⃣ 가을철 **전선·저기압 활동 강화**  
- **특징**: 여름철에 비해 대기의 온도 구배(온도 차)가 크게 형성되면서 한랭 전선(차가운 전선)과 온난 전선이 활발히 움직입니다.  
- **주요 현상**:  
  - **강수·폭우**: 전선이 통과하면서 급격한 강수와 국지적인 폭우가 발생합니다.  
  - **대기 불안정성**: 전선 근처에서 상승 기류가 강해져 뇌우·돌풍이 자주 나타납니다.  
- **왜 가을에?**: 여름에 축적된 수증기가 가을에 차가운 대기와 만나면서 전선이 강해지고, 남쪽에서 북쪽으로 이동하는 저기압(예: ‘가을 저기압’)이 빈번해집니다.

## 2️⃣ **가을철 대기역학적 현상 – ‘제트 스트림’ 강화**  
- **특징**: 겨울에 비해 아직 완전히 남쪽으로 내려가지 않은 제트 스트림이 가을에 북위 30°~50° 사이에서 가장 활발히 흐릅니다.  
- **주요 현상**:  
  - **고속 편서풍**: 제트 스트림이 강해지면 고속 편서풍이 발생해 기상 시스템(전선·저기압)의 이동 속도가 빨라집니다.  
  - **날씨 급변**: 제트 스트림에 의해 한반도·동아시아 지역은 급격히 날씨가 변하고, ‘날씨 급변 현상’(예: 갑작스러운 비·눈·바람)으로 이어집니다.  
- **왜 가을에?

In [14]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# Step 1: 현재 계절 결정
season = get_current_season("north")  # 계절 값 얻기
print(f"현재 계절: {season}")

# Step 2: 해당 계절의 자연 현상 추천
prompt2 = ChatPromptTemplate.from_template(
    "{season}에 주로 발생하는 대표적인 지구과학 현상 3가지를 알려주세요. "
    "각 현상에 대해 간단한 설명을 포함해주세요."
)

# OpenAI 모델 사용
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.0
)

# 체인 2: 자연 현상 추천 (입력: 계절 → 출력: 자연 현상 목록)
chain2 = (
    {"season": lambda x : season}  # chain1의 출력을 season 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)

# 실행: 현재 계절에 따른 자연 현상 추천
response = chain2.invoke({})
print(f"\n {season}에 발생하는 자연 현상:\n{response}")

현재 계절: 가을

 가을에 발생하는 자연 현상:
1.  **감귤 낙과** *   설명: 제주에서 감귤 낙과가 가장 성행하는 계절은 9월 초, 가을입니다.
2.  **가을 남풍** *   설명: 가을에는 서풍이 불어야 하지만, 기압의 배치 때문에 남풍이 불기도 합니다. 이 남풍은 더운 공기와 차가운 공기가 섞이면서 발생하는 기상 현상입니다.
3.  **해무리** *   설명: 가을에는 해가 지기 전이나 뜨기 전/하늘이 맑고 습기가 적을 때, 광학 현상으로 인해 해 주변에 무지개 색이 도는 무리를 관찰할 수 있습니다. 이를 일반적으로 해무리라고 합니다. 이 현상은 태양의 빛이 공기 중의 물방울에 반사되면서 발생하는 결과입니다.


#### 5-2) PartialPrompt 
* API 호출 데이터, 시간 정보, 사용자 정보 등을 반영할 때 매우 유용함

In [3]:
import requests
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 실시간 환율을 가져오는 함수
def get_exchange_rate():
    response = requests.get("https://api.exchangerate-api.com/v4/latest/USD")
    data = response.json()
    return f"1달러 = {data['rates']['KRW']}원"

# Partial Prompt 활용
prompt = PromptTemplate(
    template="현재 {info} 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.",
    input_variables=[],  # 사용자 입력 없음
    partial_variables={"info": get_exchange_rate()}  # API에서 가져온 데이터 자동 반영
)

# LLM 모델 설정
#llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# 모델에 프롬프트 전달 및 응답 받기
response = llm.invoke(prompt.format())

# 결과 출력
print(" 프롬프트:", prompt.format())
print(" 모델 응답:", response.content)

BadRequestError: Error code: 400 - {'error': {'message': 'Organization has been restricted. Please reach out to support if you believe this was in error.', 'type': 'invalid_request_error', 'code': 'organization_restricted'}}